In [2]:
using JuMP
using Printf
import Juniper
import Ipopt

a = [2.5,-2.5,2,-2.5] # x-cordinates of selected circle
b = [0.3,0.25,-2.5,-2.25] # y-cordinates of selected circle
r = [2, 2, 2, 2 ] # Radius of selected cirlcles

NumVar = size(r)[1] 

rho = 1.0  ### Augmented Lagrangian penalty coefficient
  ## Note: rho may be allowed to vary under restriction


## Typing of parameters is optional, but wise.
## Form the AL primal subproblem
function solveSP(i::Int64, vdual::Dict{String,Float64}, X::Float64,Y::Float64, R::Float64 )

  model = Model(
        optimizer_with_attributes(
            #Juniper.Optimizer,
            #"nl_solver" => optimizer_with_attributes(
                Ipopt.Optimizer, 
            #    MOI.Silent() => true,
            #),
        ),
    )
    set_optimizer_attribute(model, "print_level", 0)  ### Suppress Ipopt output
    set_optimizer_attribute(model, "tol", 1e-14)  ### Not required in general

    @variable(model, x )
    @variable(model, y )
    @variable(model, z >= 0)
  
    ### primal objective + Lagrange terms + augmented (squared penalty) terms
    @objective(model, Min, z+r[i] 
	+ vdual["x"]*(x + X - a[i]) + vdual["y"]*(y + Y - b[i]) + vdual["z"]*(z + r[i] - R)
	+ 0.5*rho*( (x+X-a[i])^2 + (y+Y-b[i])^2 + (z+r[i]-R)^2 ) )

    ### All coupling constraints are relaxed.
    @constraint( model, xyz, x^2 + y^2 <= z^2)
  
    #@show model
    
    #print(model)
    optimize!(model)

    return value(x), value(y), value(z)
end

function main()
  ## Initialization
    x = Dict{Int64,Float64}()
    y = Dict{Int64,Float64}()
    z = Dict{Int64,Float64}()

    ### Initialize Lagrangian multipliers to zero value
    vdual = Dict{Int64, Dict{String,Float64}}()
    for i in 1:NumVar
	vdual[i] = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    end

    X,Y,R=0.0,0.0,0.0
    oldX,oldY,oldR=0.0,0.0,0.0
    ### Initialize R to average of r[i], i=1,...,n
    for i in 1:length(a)
	R += (1.0/NumVar)*(r[i])
    end
  ## End of initialization

for n in 1:1000
    ### Solve AL subproblem for each subproblem i with X,Y,R fixed
    for i in 1:NumVar
      x[i],y[i],z[i] = solveSP(i,vdual[i],X,Y,R)
    end

    ### Subproblems in X,Y,R (with x,y,z fixed) can be solved in closed-form
    oldX,oldY,oldR=X,Y,R
    X,Y,R=0.0,0.0,0.0
    for i in 1:NumVar
	X += (1.0/NumVar)*(a[i]-x[i])
	Y += (1.0/NumVar)*(b[i]-y[i])
	R += (1.0/NumVar)*(z[i]+r[i])
    end

    ### Update Lagrange multipliers, compute a measure of primal infeasibility.
    discr = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    for i in 1:NumVar
	vdual[i]["x"] += rho*(x[i] + X - a[i]) 
	vdual[i]["y"] += rho*(y[i] + Y - b[i]) 
	vdual[i]["z"] += rho*(z[i] + r[i] - R)

	discr["x"] += abs(x[i] + X - a[i])
	discr["y"] += abs(y[i] + Y - b[i])
	discr["z"] += abs(z[i] + r[i] - R)
    end
    @printf("Iter %d Primal Discrepancies %8.8f R value %8.8f\n", 
	n, discr["x"] + discr["y"] + discr["z"], R)
    d_discr = abs(X-oldX)+abs(Y-oldY)+abs(R-oldR) ### Dual residues
    if discr["x"] + discr["y"] + discr["z"] + d_discr < 1e-7
	println("Stopping at iteration ",n)  ### Primal feasible within tolerance
	break
    end
end
    @show(X)
    @show(Y)
    @show(R)

println("Correct values are: ")
    
println("X = ",-1.0456187885555454e-7)
println("Y = ",-0.9749997949767099)
println("R = ",4.806354399951325)
end

main()

Iter 1 Primal Discrepancies 10.67947466 R value 2.94942158
Iter 2 Primal Discrepancies 3.07165476 R value 4.26249368
Iter 3 Primal Discrepancies 2.58813419 R value 5.14531200
Iter 4 Primal Discrepancies 3.74587354 R value 5.37591350
Iter 5 Primal Discrepancies 2.57022246 R value 5.16629049
Iter 6 Primal Discrepancies 0.78953482 R value 4.84267703
Iter 7 Primal Discrepancies 0.71916549 R value 4.62508178
Iter 8 Primal Discrepancies 1.02607066 R value 4.56982153
Iter 9 Primal Discrepancies 0.73936137 R value 4.62338513
Iter 10 Primal Discrepancies 0.36149338 R value 4.70468649
Iter 11 Primal Discrepancies 0.36266554 R value 4.75962469
Iter 12 Primal Discrepancies 0.37986607 R value 4.77455470
Iter 13 Primal Discrepancies 0.35949552 R value 4.76268308
Iter 14 Primal Discrepancies 0.35246401 R value 4.74384374
Iter 15 Primal Discrepancies 0.35068271 R value 4.73116899
Iter 16 Primal Discrepancies 0.35075186 R value 4.72790603
Iter 17 Primal Discrepancies 0.35188219 R value 4.73084141
Iter 

In [3]:
##QP_ADMM for 4 selected circles in n=10, m=25 test case

using JuMP
using Printf
import Juniper
import Ipopt

a = [46.89, 18.36, 24.81, 14.31, 45.81]
b = [34.21, 34.21, 15.03, 5.03, 17.23]
r = [20, 20, 20, 20, 20 ]
NumVar = size(r)[1] 

rho = 1.0  ### Augmented Lagrangian penalty coefficient
  ## Note: rho may be allowed to vary under restriction


## Typing of parameters is optional, but wise.
## Form the AL primal subproblem
function solveSP(i::Int64, vdual::Dict{String,Float64}, X::Float64,Y::Float64, R::Float64 )

  model = Model(
        optimizer_with_attributes(
            #Juniper.Optimizer,
            #"nl_solver" => optimizer_with_attributes(
                Ipopt.Optimizer, 
            #    MOI.Silent() => true,
            #),
        ),
    )
    set_optimizer_attribute(model, "print_level", 0)  ### Suppress Ipopt output
    set_optimizer_attribute(model, "tol", 1e-14)  ### Not required in general

    @variable(model, x )
    @variable(model, y )
    @variable(model, z >= 0)
  
    ### primal objective + Lagrange terms + augmented (squared penalty) terms
    @objective(model, Min, z+r[i] 
	+ vdual["x"]*(x + X - a[i]) + vdual["y"]*(y + Y - b[i]) + vdual["z"]*(z + r[i] - R)
	+ 0.5*rho*( (x+X-a[i])^2 + (y+Y-b[i])^2 + (z+r[i]-R)^2 ) )

    ### All coupling constraints are relaxed.
    @constraint( model, xyz, x^2 + y^2 <= z^2)
  
    #@show model
    
    #print(model)
    optimize!(model)

    return value(x), value(y), value(z)
end

function main()
  ## Initialization
    x = Dict{Int64,Float64}()
    y = Dict{Int64,Float64}()
    z = Dict{Int64,Float64}()

    ### Initialize Lagrangian multipliers to zero value
    vdual = Dict{Int64, Dict{String,Float64}}()
    for i in 1:NumVar
	vdual[i] = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    end

    X,Y,R=0.0,0.0,0.0
    oldX,oldY,oldR=0.0,0.0,0.0
    ### Initialize R to average of r[i], i=1,...,n
    for i in 1:length(a)
	R += (1.0/NumVar)*(r[i])
    end
  ## End of initialization

for n in 1:1000
    ### Solve AL subproblem for each subproblem i with X,Y,R fixed
    for i in 1:NumVar
      x[i],y[i],z[i] = solveSP(i,vdual[i],X,Y,R)
    end

    ### Subproblems in X,Y,R (with x,y,z fixed) can be solved in closed-form
    oldX,oldY,oldR=X,Y,R
    X,Y,R=0.0,0.0,0.0
    for i in 1:NumVar
	X += (1.0/NumVar)*(a[i]-x[i])
	Y += (1.0/NumVar)*(b[i]-y[i])
	R += (1.0/NumVar)*(z[i]+r[i])
    end

    ### Update Lagrange multipliers, compute a measure of primal infeasibility.
    discr = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    for i in 1:NumVar
	vdual[i]["x"] += rho*(x[i] + X - a[i]) 
	vdual[i]["y"] += rho*(y[i] + Y - b[i]) 
	vdual[i]["z"] += rho*(z[i] + r[i] - R)

	discr["x"] += abs(x[i] + X - a[i])
	discr["y"] += abs(y[i] + Y - b[i])
	discr["z"] += abs(z[i] + r[i] - R)
    end
    @printf("Iter %d Primal Discrepancies %8.8f R value %8.8f\n", 
	n, discr["x"] + discr["y"] + discr["z"], R)
    d_discr = abs(X-oldX)+abs(Y-oldY)+abs(R-oldR) ### Dual residues
    if discr["x"] + discr["y"] + discr["z"] + d_discr < 1e-7
	println("Stopping at iteration ",n)  ### Primal feasible within tolerance
	break
    end
end
    @show(X)
    @show(Y)
    @show(R)

println("Correct values are: ")

println("X = ",30.599999991460297)
println("Y = ",19.62000000953417)
println("R = ",41.868520759503134)
end

main()

Iter 1 Primal Discrepancies 91.12239784 R value 38.49874235
Iter 2 Primal Discrepancies 33.42531768 R value 45.71044490
Iter 3 Primal Discrepancies 36.76520860 R value 49.35202882
Iter 4 Primal Discrepancies 29.63002468 R value 50.80314363
Iter 5 Primal Discrepancies 18.39572260 R value 51.29969065
Iter 6 Primal Discrepancies 18.53104863 R value 50.83332874
Iter 7 Primal Discrepancies 10.26656969 R value 49.83332874
Iter 8 Primal Discrepancies 0.00000000 R value 48.83332874
Iter 9 Primal Discrepancies 0.00000000 R value 47.83332874
Iter 10 Primal Discrepancies 0.00000000 R value 46.83332874
Iter 11 Primal Discrepancies 0.00000000 R value 45.83332874
Iter 12 Primal Discrepancies 0.00000000 R value 44.83332874
Iter 13 Primal Discrepancies 0.00000000 R value 43.83332874
Iter 14 Primal Discrepancies 0.57090713 R value 42.86300505
Iter 15 Primal Discrepancies 2.26632455 R value 42.01048716
Iter 16 Primal Discrepancies 3.16939357 R value 41.32271748
Iter 17 Primal Discrepancies 3.67676846 R 

Iter 171 Primal Discrepancies 0.00002299 R value 41.86852079
Iter 172 Primal Discrepancies 0.00002198 R value 41.86852081
Iter 173 Primal Discrepancies 0.00002102 R value 41.86852081
Iter 174 Primal Discrepancies 0.00002011 R value 41.86852081
Iter 175 Primal Discrepancies 0.00001922 R value 41.86852081
Iter 176 Primal Discrepancies 0.00001837 R value 41.86852080
Iter 177 Primal Discrepancies 0.00001756 R value 41.86852078
Iter 178 Primal Discrepancies 0.00001678 R value 41.86852077
Iter 179 Primal Discrepancies 0.00001605 R value 41.86852076
Iter 180 Primal Discrepancies 0.00001535 R value 41.86852076
Iter 181 Primal Discrepancies 0.00001468 R value 41.86852075
Iter 182 Primal Discrepancies 0.00001403 R value 41.86852076
Iter 183 Primal Discrepancies 0.00001341 R value 41.86852076
Iter 184 Primal Discrepancies 0.00001282 R value 41.86852076
Iter 185 Primal Discrepancies 0.00001226 R value 41.86852077
Iter 186 Primal Discrepancies 0.00001172 R value 41.86852077
Iter 187 Primal Discrepa

In [4]:
##QP_ADMM for 4 selected circles in n=10, m=50 test case

using JuMP
using Printf
import Juniper
import Ipopt

a = [13.239512466541303, 5.513695828127543, 41.55335924184264, 43.70990076229758] # x-cordinates of selected circle
b = [43.945805488954605, 14.997506943080591, 31.204816928920724, 15.106911655793608] # y-cordinates of selected circle
r = [20, 20, 20, 20 ] # Radius of selected cirlcles

NumVar = size(r)[1] 

rho = 1.0  ### Augmented Lagrangian penalty coefficient
  ## Note: rho may be allowed to vary under restriction


## Typing of parameters is optional, but wise.
## Form the AL primal subproblem
function solveSP(i::Int64, vdual::Dict{String,Float64}, X::Float64,Y::Float64, R::Float64 )

  model = Model(
        optimizer_with_attributes(
            #Juniper.Optimizer,
            #"nl_solver" => optimizer_with_attributes(
                Ipopt.Optimizer, 
            #    MOI.Silent() => true,
            #),
        ),
    )
    set_optimizer_attribute(model, "print_level", 0)  ### Suppress Ipopt output
    set_optimizer_attribute(model, "tol", 1e-14)  ### Not required in general

    @variable(model, x )
    @variable(model, y )
    @variable(model, z >= 0)
  
    ### primal objective + Lagrange terms + augmented (squared penalty) terms
    @objective(model, Min, z+r[i] 
	+ vdual["x"]*(x + X - a[i]) + vdual["y"]*(y + Y - b[i]) + vdual["z"]*(z + r[i] - R)
	+ 0.5*rho*( (x+X-a[i])^2 + (y+Y-b[i])^2 + (z+r[i]-R)^2 ) )

    ### All coupling constraints are relaxed.
    @constraint( model, xyz, x^2 + y^2 <= z^2)
  
    #@show model
    
    #print(model)
    optimize!(model)

    return value(x), value(y), value(z)
end

function main()
  ## Initialization
    x = Dict{Int64,Float64}()
    y = Dict{Int64,Float64}()
    z = Dict{Int64,Float64}()

    ### Initialize Lagrangian multipliers to zero value
    vdual = Dict{Int64, Dict{String,Float64}}()
    for i in 1:NumVar
	vdual[i] = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    end

    X,Y,R=0.0,0.0,0.0
    oldX,oldY,oldR=0.0,0.0,0.0
    ### Initialize R to average of r[i], i=1,...,n
    for i in 1:length(a)
	R += (1.0/NumVar)*(r[i])
    end
  ## End of initialization

for n in 1:1000
    ### Solve AL subproblem for each subproblem i with X,Y,R fixed
    for i in 1:NumVar
      x[i],y[i],z[i] = solveSP(i,vdual[i],X,Y,R)
    end

    ### Subproblems in X,Y,R (with x,y,z fixed) can be solved in closed-form
    oldX,oldY,oldR=X,Y,R
    X,Y,R=0.0,0.0,0.0
    for i in 1:NumVar
	X += (1.0/NumVar)*(a[i]-x[i])
	Y += (1.0/NumVar)*(b[i]-y[i])
	R += (1.0/NumVar)*(z[i]+r[i])
    end

    ### Update Lagrange multipliers, compute a measure of primal infeasibility.
    discr = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    for i in 1:NumVar
	vdual[i]["x"] += rho*(x[i] + X - a[i]) 
	vdual[i]["y"] += rho*(y[i] + Y - b[i]) 
	vdual[i]["z"] += rho*(z[i] + r[i] - R)

	discr["x"] += abs(x[i] + X - a[i])
	discr["y"] += abs(y[i] + Y - b[i])
	discr["z"] += abs(z[i] + r[i] - R)
    end
    @printf("Iter %d Primal Discrepancies %8.8f R value %8.8f\n", 
	n, discr["x"] + discr["y"] + discr["z"], R)
    d_discr = abs(X-oldX)+abs(Y-oldY)+abs(R-oldR) ### Dual residues
    if discr["x"] + discr["y"] + discr["z"] + d_discr < 1e-7
	println("Stopping at iteration ",n)  ### Primal feasible within tolerance
	break
    end
end
    @show(X)
    @show(Y)
    @show(R)

println("Correct values are: ")

println("X = ",24.58212046374567)
println("Y = ",25.41355826361457)
println("R = ",41.7278379893865)
end

main()

Iter 1 Primal Discrepancies 80.52325487 R value 39.51102756
Iter 2 Primal Discrepancies 16.83723797 R value 48.18803002
Iter 3 Primal Discrepancies 39.21615848 R value 50.94697438
Iter 4 Primal Discrepancies 25.29154790 R value 50.19679792
Iter 5 Primal Discrepancies 3.44454324 R value 49.19679792
Iter 6 Primal Discrepancies 0.00000000 R value 48.19679792
Iter 7 Primal Discrepancies 0.00000000 R value 47.19679792
Iter 8 Primal Discrepancies 0.00000000 R value 46.19679792
Iter 9 Primal Discrepancies 0.00000000 R value 45.19679792
Iter 10 Primal Discrepancies 1.52529031 R value 44.31000895
Iter 11 Primal Discrepancies 2.50015802 R value 43.71474007
Iter 12 Primal Discrepancies 2.51773515 R value 43.43900606
Iter 13 Primal Discrepancies 1.84582768 R value 43.33683101
Iter 14 Primal Discrepancies 1.48890836 R value 43.26405883
Iter 15 Primal Discrepancies 1.22735621 R value 43.19922197
Iter 16 Primal Discrepancies 1.18293211 R value 43.16399602
Iter 17 Primal Discrepancies 0.98436373 R val

Iter 171 Primal Discrepancies 0.00000084 R value 41.72783794
Iter 172 Primal Discrepancies 0.00000581 R value 41.72783746
Iter 173 Primal Discrepancies 0.00001024 R value 41.72783711
Iter 174 Primal Discrepancies 0.00001292 R value 41.72783690
Iter 175 Primal Discrepancies 0.00001380 R value 41.72783685
Iter 176 Primal Discrepancies 0.00001306 R value 41.72783692
Iter 177 Primal Discrepancies 0.00001101 R value 41.72783710
Iter 178 Primal Discrepancies 0.00000806 R value 41.72783735
Iter 179 Primal Discrepancies 0.00000466 R value 41.72783763
Iter 180 Primal Discrepancies 0.00000132 R value 41.72783791
Iter 181 Primal Discrepancies 0.00000179 R value 41.72783815
Iter 182 Primal Discrepancies 0.00000420 R value 41.72783834
Iter 183 Primal Discrepancies 0.00000580 R value 41.72783847
Iter 184 Primal Discrepancies 0.00000654 R value 41.72783852
Iter 185 Primal Discrepancies 0.00000646 R value 41.72783851
Iter 186 Primal Discrepancies 0.00000569 R value 41.72783844
Iter 187 Primal Discrepa

In [5]:
##QP_ADMM for 4 selected circles in n=10, m=100 test case

using JuMP
using Printf
import Juniper
import Ipopt

a = [13.239512466541303, 41.55335924184264, 43.70990076229758, 13.150886893355297] # x-cordinates of selected circle
b = [43.945805488954605, 31.204816928920724, 15.106911655793608, 11.789181775998983] # y-cordinates of selected circle
r = [20, 20, 20, 20 ] # Radius of selected cirlcles
   

NumVar = size(r)[1] 

rho = 1.0  ### Augmented Lagrangian penalty coefficient
  ## Note: rho may be allowed to vary under restriction


## Typing of parameters is optional, but wise.
## Form the AL primal subproblem
function solveSP(i::Int64, vdual::Dict{String,Float64}, X::Float64,Y::Float64, R::Float64 )

  model = Model(
        optimizer_with_attributes(
            #Juniper.Optimizer,
            #"nl_solver" => optimizer_with_attributes(
                Ipopt.Optimizer, 
            #    MOI.Silent() => true,
            #),
        ),
    )
    set_optimizer_attribute(model, "print_level", 0)  ### Suppress Ipopt output
    set_optimizer_attribute(model, "tol", 1e-14)  ### Not required in general

    @variable(model, x )
    @variable(model, y )
    @variable(model, z >= 0)
  
    ### primal objective + Lagrange terms + augmented (squared penalty) terms
    @objective(model, Min, z+r[i] 
	+ vdual["x"]*(x + X - a[i]) + vdual["y"]*(y + Y - b[i]) + vdual["z"]*(z + r[i] - R)
	+ 0.5*rho*( (x+X-a[i])^2 + (y+Y-b[i])^2 + (z+r[i]-R)^2 ) )

    ### All coupling constraints are relaxed.
    @constraint( model, xyz, x^2 + y^2 <= z^2)
  
    #@show model
    
    #print(model)
    optimize!(model)

    return value(x), value(y), value(z)
end

function main()
  ## Initialization
    x = Dict{Int64,Float64}()
    y = Dict{Int64,Float64}()
    z = Dict{Int64,Float64}()

    ### Initialize Lagrangian multipliers to zero value
    vdual = Dict{Int64, Dict{String,Float64}}()
    for i in 1:NumVar
	vdual[i] = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    end

    X,Y,R=0.0,0.0,0.0
    oldX,oldY,oldR=0.0,0.0,0.0
    ### Initialize R to average of r[i], i=1,...,n
    for i in 1:length(a)
	R += (1.0/NumVar)*(r[i])
    end
  ## End of initialization

for n in 1:1000
    ### Solve AL subproblem for each subproblem i with X,Y,R fixed
    for i in 1:NumVar
      x[i],y[i],z[i] = solveSP(i,vdual[i],X,Y,R)
    end

    ### Subproblems in X,Y,R (with x,y,z fixed) can be solved in closed-form
    oldX,oldY,oldR=X,Y,R
    X,Y,R=0.0,0.0,0.0
    for i in 1:NumVar
	X += (1.0/NumVar)*(a[i]-x[i])
	Y += (1.0/NumVar)*(b[i]-y[i])
	R += (1.0/NumVar)*(z[i]+r[i])
    end

    ### Update Lagrange multipliers, compute a measure of primal infeasibility.
    discr = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    for i in 1:NumVar
	vdual[i]["x"] += rho*(x[i] + X - a[i]) 
	vdual[i]["y"] += rho*(y[i] + Y - b[i]) 
	vdual[i]["z"] += rho*(z[i] + r[i] - R)

	discr["x"] += abs(x[i] + X - a[i])
	discr["y"] += abs(y[i] + Y - b[i])
	discr["z"] += abs(z[i] + r[i] - R)
    end
    @printf("Iter %d Primal Discrepancies %8.8f R value %8.8f\n", 
	n, discr["x"] + discr["y"] + discr["z"], R)
    d_discr = abs(X-oldX)+abs(Y-oldY)+abs(R-oldR) ### Dual residues
    if discr["x"] + discr["y"] + discr["z"] + d_discr < 1e-7
	println("Stopping at iteration ",n)  ### Primal feasible within tolerance
	break
    end
end
    @show(X)
    @show(Y)
    @show(R)

println("Correct values are: ")

println("X = ",26.868995309890536)
println("Y = ",27.829807823090338)
println("R = ",41.1065909982292)
end

main()

Iter 1 Primal Discrepancies 76.98261271 R value 39.72135665
Iter 2 Primal Discrepancies 14.80626207 R value 47.97197368
Iter 3 Primal Discrepancies 37.32830971 R value 50.64658934
Iter 4 Primal Discrepancies 23.67589522 R value 50.09443235
Iter 5 Primal Discrepancies 6.16034356 R value 49.09443235
Iter 6 Primal Discrepancies 0.00000000 R value 48.09443235
Iter 7 Primal Discrepancies 0.00000000 R value 47.09443235
Iter 8 Primal Discrepancies 0.00000000 R value 46.09443235
Iter 9 Primal Discrepancies 0.00000000 R value 45.09443235
Iter 10 Primal Discrepancies 1.43927397 R value 44.20135298
Iter 11 Primal Discrepancies 2.54472106 R value 43.64747754
Iter 12 Primal Discrepancies 2.48028700 R value 43.40911201
Iter 13 Primal Discrepancies 1.78938782 R value 43.32020003
Iter 14 Primal Discrepancies 1.51733579 R value 43.24131245
Iter 15 Primal Discrepancies 1.11110271 R value 43.17494542
Iter 16 Primal Discrepancies 1.00255863 R value 43.13811256
Iter 17 Primal Discrepancies 0.81589395 R val

Iter 171 Primal Discrepancies 0.00010028 R value 41.10657470
Iter 172 Primal Discrepancies 0.00006604 R value 41.10658045
Iter 173 Primal Discrepancies 0.00003029 R value 41.10658641
Iter 174 Primal Discrepancies 0.00000607 R value 41.10659201
Iter 175 Primal Discrepancies 0.00003263 R value 41.10659679
Iter 176 Primal Discrepancies 0.00005498 R value 41.10660041
Iter 177 Primal Discrepancies 0.00006943 R value 41.10660271
Iter 178 Primal Discrepancies 0.00007572 R value 41.10660365
Iter 179 Primal Discrepancies 0.00007433 R value 41.10660333
Iter 180 Primal Discrepancies 0.00006640 R value 41.10660192
Iter 181 Primal Discrepancies 0.00005350 R value 41.10659972
Iter 182 Primal Discrepancies 0.00003743 R value 41.10659701
Iter 183 Primal Discrepancies 0.00002008 R value 41.10659411
Iter 184 Primal Discrepancies 0.00000462 R value 41.10659130
Iter 185 Primal Discrepancies 0.00001181 R value 41.10658884
Iter 186 Primal Discrepancies 0.00002378 R value 41.10658689
Iter 187 Primal Discrepa

In [6]:
##QP_ADMM for 3 selected circles in n=20, m=25 test case

using JuMP
using Printf
import Juniper
import Ipopt

a = [49.42043424128368, 31.611086580497098,  12.419857083265606] # x-cordinates of selected circle
b = [ 44.79128960675432, 19.19929956530496, 28.300299268064713] # y-cordinates of selected circle
r = [20, 20, 20 ] # Radius of selected cirlcles
   
   

NumVar = size(r)[1] 

rho = 1.0  ### Augmented Lagrangian penalty coefficient
  ## Note: rho may be allowed to vary under restriction


## Typing of parameters is optional, but wise.
## Form the AL primal subproblem
function solveSP(i::Int64, vdual::Dict{String,Float64}, X::Float64,Y::Float64, R::Float64 )

  model = Model(
        optimizer_with_attributes(
            #Juniper.Optimizer,
            #"nl_solver" => optimizer_with_attributes(
                Ipopt.Optimizer, 
            #    MOI.Silent() => true,
            #),
        ),
    )
    set_optimizer_attribute(model, "print_level", 0)  ### Suppress Ipopt output
    set_optimizer_attribute(model, "tol", 1e-14)  ### Not required in general

    @variable(model, x )
    @variable(model, y )
    @variable(model, z >= 0)
  
    ### primal objective + Lagrange terms + augmented (squared penalty) terms
    @objective(model, Min, z+r[i] 
	+ vdual["x"]*(x + X - a[i]) + vdual["y"]*(y + Y - b[i]) + vdual["z"]*(z + r[i] - R)
	+ 0.5*rho*( (x+X-a[i])^2 + (y+Y-b[i])^2 + (z+r[i]-R)^2 ) )

    ### All coupling constraints are relaxed.
    @constraint( model, xyz, x^2 + y^2 <= z^2)
  
    #@show model
    
    #print(model)
    optimize!(model)

    return value(x), value(y), value(z)
end

function main()
  ## Initialization
    x = Dict{Int64,Float64}()
    y = Dict{Int64,Float64}()
    z = Dict{Int64,Float64}()

    ### Initialize Lagrangian multipliers to zero value
    vdual = Dict{Int64, Dict{String,Float64}}()
    for i in 1:NumVar
	vdual[i] = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    end

    X,Y,R=0.0,0.0,0.0
    oldX,oldY,oldR=0.0,0.0,0.0
    ### Initialize R to average of r[i], i=1,...,n
    for i in 1:length(a)
	R += (1.0/NumVar)*(r[i])
    end
  ## End of initialization

for n in 1:1000
    ### Solve AL subproblem for each subproblem i with X,Y,R fixed
    for i in 1:NumVar
      x[i],y[i],z[i] = solveSP(i,vdual[i],X,Y,R)
    end

    ### Subproblems in X,Y,R (with x,y,z fixed) can be solved in closed-form
    oldX,oldY,oldR=X,Y,R
    X,Y,R=0.0,0.0,0.0
    for i in 1:NumVar
	X += (1.0/NumVar)*(a[i]-x[i])
	Y += (1.0/NumVar)*(b[i]-y[i])
	R += (1.0/NumVar)*(z[i]+r[i])
    end

    ### Update Lagrange multipliers, compute a measure of primal infeasibility.
    discr = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    for i in 1:NumVar
	vdual[i]["x"] += rho*(x[i] + X - a[i]) 
	vdual[i]["y"] += rho*(y[i] + Y - b[i]) 
	vdual[i]["z"] += rho*(z[i] + r[i] - R)

	discr["x"] += abs(x[i] + X - a[i])
	discr["y"] += abs(y[i] + Y - b[i])
	discr["z"] += abs(z[i] + r[i] - R)
    end
    @printf("Iter %d Primal Discrepancies %8.8f R value %8.8f\n", 
	n, discr["x"] + discr["y"] + discr["z"], R)
    d_discr = abs(X-oldX)+abs(Y-oldY)+abs(R-oldR) ### Dual residues
    if discr["x"] + discr["y"] + discr["z"] + d_discr < 1e-7
	println("Stopping at iteration ",n)  ### Primal feasible within tolerance
	break
    end
end
    @show(X)
    @show(Y)
    @show(R)

println("Correct values are: ")

println("X = ",30.920145668409557)
println("Y = ",36.54579442364673)
println("R = ",40.254601163366)
end

main()

Iter 1 Primal Discrepancies 54.82693714 R value 41.93142613
Iter 2 Primal Discrepancies 28.68245640 R value 48.57508480
Iter 3 Primal Discrepancies 9.84631065 R value 54.19366122
Iter 4 Primal Discrepancies 26.24570954 R value 57.08953309
Iter 5 Primal Discrepancies 28.41912126 R value 57.03833444
Iter 6 Primal Discrepancies 9.15019189 R value 56.03833444
Iter 7 Primal Discrepancies 0.00000000 R value 55.03833444
Iter 8 Primal Discrepancies 0.00000000 R value 54.03833444
Iter 9 Primal Discrepancies 0.00000000 R value 53.03833444
Iter 10 Primal Discrepancies 0.00000000 R value 52.03833444
Iter 11 Primal Discrepancies 0.00000000 R value 51.03833444
Iter 12 Primal Discrepancies 0.00000000 R value 50.03833444
Iter 13 Primal Discrepancies 0.00000000 R value 49.03833444
Iter 14 Primal Discrepancies 0.00000000 R value 48.03833444
Iter 15 Primal Discrepancies 0.00000000 R value 47.03833444
Iter 16 Primal Discrepancies 0.00000000 R value 46.03833444
Iter 17 Primal Discrepancies 0.34032453 R val

Iter 171 Primal Discrepancies 0.00003735 R value 40.25460196
Iter 172 Primal Discrepancies 0.00003559 R value 40.25460189
Iter 173 Primal Discrepancies 0.00003391 R value 40.25460182
Iter 174 Primal Discrepancies 0.00003231 R value 40.25460176
Iter 175 Primal Discrepancies 0.00003079 R value 40.25460171
Iter 176 Primal Discrepancies 0.00002934 R value 40.25460166
Iter 177 Primal Discrepancies 0.00002796 R value 40.25460161
Iter 178 Primal Discrepancies 0.00002664 R value 40.25460157
Iter 179 Primal Discrepancies 0.00002538 R value 40.25460153
Iter 180 Primal Discrepancies 0.00002419 R value 40.25460150
Iter 181 Primal Discrepancies 0.00002305 R value 40.25460147
Iter 182 Primal Discrepancies 0.00002196 R value 40.25460144
Iter 183 Primal Discrepancies 0.00002093 R value 40.25460141
Iter 184 Primal Discrepancies 0.00001994 R value 40.25460139
Iter 185 Primal Discrepancies 0.00001900 R value 40.25460137
Iter 186 Primal Discrepancies 0.00001811 R value 40.25460135
Iter 187 Primal Discrepa

Iter 339 Primal Discrepancies 0.00000001 R value 40.25460116
Iter 340 Primal Discrepancies 0.00000001 R value 40.25460116
Iter 341 Primal Discrepancies 0.00000001 R value 40.25460116
Iter 342 Primal Discrepancies 0.00000001 R value 40.25460116
Iter 343 Primal Discrepancies 0.00000001 R value 40.25460116
Iter 344 Primal Discrepancies 0.00000001 R value 40.25460116
Stopping at iteration 344
X = 30.920146220602106
Y = 36.54579318469892
R = 40.254601158361574
Correct values are: 
X = 30.920145668409557
Y = 36.54579442364673
R = 40.254601163366


In [7]:
##QP_ADMM for 4 selected circles in n=20, m=50 test case

using JuMP
using Printf
import Juniper
import Ipopt

a = [8.952575686584531, 46.74153578924209, 32.34771758225827, 11.355837839937607] # x-cordinates of selected circle
b = [38.87977889206978, 23.45904720270292, 37.565559688461505, 13.734380115583644] # y-cordinates of selected circle
r = [20, 20, 20, 20 ] # Radius of selected circles
   
   

NumVar = size(r)[1] 

rho = 1.0  ### Augmented Lagrangian penalty coefficient
  ## Note: rho may be allowed to vary under restriction


## Typing of parameters is optional, but wise.
## Form the AL primal subproblem
function solveSP(i::Int64, vdual::Dict{String,Float64}, X::Float64,Y::Float64, R::Float64 )

  model = Model(
        optimizer_with_attributes(
            #Juniper.Optimizer,
            #"nl_solver" => optimizer_with_attributes(
                Ipopt.Optimizer, 
            #    MOI.Silent() => true,
            #),
        ),
    )
    set_optimizer_attribute(model, "print_level", 0)  ### Suppress Ipopt output
    set_optimizer_attribute(model, "tol", 1e-14)  ### Not required in general

    @variable(model, x )
    @variable(model, y )
    @variable(model, z >= 0)
  
    ### primal objective + Lagrange terms + augmented (squared penalty) terms
    @objective(model, Min, z+r[i] 
	+ vdual["x"]*(x + X - a[i]) + vdual["y"]*(y + Y - b[i]) + vdual["z"]*(z + r[i] - R)
	+ 0.5*rho*( (x+X-a[i])^2 + (y+Y-b[i])^2 + (z+r[i]-R)^2 ) )

    ### All coupling constraints are relaxed.
    @constraint( model, xyz, x^2 + y^2 <= z^2)
  
    #@show model
    
    #print(model)
    optimize!(model)

    return value(x), value(y), value(z)
end

function main()
  ## Initialization
    x = Dict{Int64,Float64}()
    y = Dict{Int64,Float64}()
    z = Dict{Int64,Float64}()

    ### Initialize Lagrangian multipliers to zero value
    vdual = Dict{Int64, Dict{String,Float64}}()
    for i in 1:NumVar
	vdual[i] = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    end

    X,Y,R=0.0,0.0,0.0
    oldX,oldY,oldR=0.0,0.0,0.0
    ### Initialize R to average of r[i], i=1,...,n
    for i in 1:length(a)
	R += (1.0/NumVar)*(r[i])
    end
  ## End of initialization

for n in 1:1000
    ### Solve AL subproblem for each subproblem i with X,Y,R fixed
    for i in 1:NumVar
      x[i],y[i],z[i] = solveSP(i,vdual[i],X,Y,R)
    end

    ### Subproblems in X,Y,R (with x,y,z fixed) can be solved in closed-form
    oldX,oldY,oldR=X,Y,R
    X,Y,R=0.0,0.0,0.0
    for i in 1:NumVar
	X += (1.0/NumVar)*(a[i]-x[i])
	Y += (1.0/NumVar)*(b[i]-y[i])
	R += (1.0/NumVar)*(z[i]+r[i])
    end

    ### Update Lagrange multipliers, compute a measure of primal infeasibility.
    discr = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    for i in 1:NumVar
	vdual[i]["x"] += rho*(x[i] + X - a[i]) 
	vdual[i]["y"] += rho*(y[i] + Y - b[i]) 
	vdual[i]["z"] += rho*(z[i] + r[i] - R)

	discr["x"] += abs(x[i] + X - a[i])
	discr["y"] += abs(y[i] + Y - b[i])
	discr["z"] += abs(z[i] + r[i] - R)
    end
    @printf("Iter %d Primal Discrepancies %8.8f R value %8.8f\n", 
	n, discr["x"] + discr["y"] + discr["z"], R)
    d_discr = abs(X-oldX)+abs(Y-oldY)+abs(R-oldR) ### Dual residues
    if discr["x"] + discr["y"] + discr["z"] + d_discr < 1e-7
	println("Stopping at iteration ",n)  ### Primal feasible within tolerance
	break
    end
end
    @show(X)
    @show(Y)
    @show(R)

println("Correct values are: ")

println("X = ",26.50038624217553)
println("Y = ",27.86935953170211)
println("R = ",40.71605633426372)
end

main()

Iter 1 Primal Discrepancies 71.69202420 R value 39.44875059
Iter 2 Primal Discrepancies 19.18066262 R value 47.11887062
Iter 3 Primal Discrepancies 36.16155969 R value 49.43045484
Iter 4 Primal Discrepancies 15.77218649 R value 50.03878864
Iter 5 Primal Discrepancies 12.45279713 R value 49.65665032
Iter 6 Primal Discrepancies 7.92289313 R value 48.65665032
Iter 7 Primal Discrepancies 0.00000000 R value 47.65665032
Iter 8 Primal Discrepancies 0.00000000 R value 46.65665032
Iter 9 Primal Discrepancies 0.00000000 R value 45.65665032
Iter 10 Primal Discrepancies 0.00000000 R value 44.65665032
Iter 11 Primal Discrepancies 0.39449678 R value 43.69168687
Iter 12 Primal Discrepancies 2.58148694 R value 42.98961309
Iter 13 Primal Discrepancies 3.30089947 R value 42.62360858
Iter 14 Primal Discrepancies 2.78400462 R value 42.54069117
Iter 15 Primal Discrepancies 1.61277370 R value 42.62152357
Iter 16 Primal Discrepancies 0.39755024 R value 42.73949463
Iter 17 Primal Discrepancies 0.56772927 R va

Iter 171 Primal Discrepancies 0.00001051 R value 40.71605780
Iter 172 Primal Discrepancies 0.00005055 R value 40.71606268
Iter 173 Primal Discrepancies 0.00008376 R value 40.71606642
Iter 174 Primal Discrepancies 0.00010593 R value 40.71606887
Iter 175 Primal Discrepancies 0.00011659 R value 40.71606998
Iter 176 Primal Discrepancies 0.00011630 R value 40.71606982
Iter 177 Primal Discrepancies 0.00010642 R value 40.71606857
Iter 178 Primal Discrepancies 0.00008896 R value 40.71606646
Iter 179 Primal Discrepancies 0.00006631 R value 40.71606377
Iter 180 Primal Discrepancies 0.00004098 R value 40.71606080
Iter 181 Primal Discrepancies 0.00001540 R value 40.71605783
Iter 182 Primal Discrepancies 0.00000853 R value 40.71605510
Iter 183 Primal Discrepancies 0.00002836 R value 40.71605281
Iter 184 Primal Discrepancies 0.00004359 R value 40.71605109
Iter 185 Primal Discrepancies 0.00005334 R value 40.71605002
Iter 186 Primal Discrepancies 0.00005747 R value 40.71604961
Iter 187 Primal Discrepa

In [9]:
##QP_ADMM for 4 selected circles in n=20, m=100 test case

using JuMP
using Printf
import Juniper
import Ipopt

a = [31.638687927784943,1.6854696772411517, 34.96903642134066, 7.147231577211446] # x-cordinates of selected circle
b = [17.17851430439765, 2.8765182589078004, 39.17787190718997, 33.785980366613536] # y-cordinates of selected circle
r = [20, 20, 20, 20 ] # Radius of selected circles
   
NumVar = size(r)[1] 

rho = 1.0  ### Augmented Lagrangian penalty coefficient
  ## Note: rho may be allowed to vary under restriction


## Typing of parameters is optional, but wise.
## Form the AL primal subproblem
function solveSP(i::Int64, vdual::Dict{String,Float64}, X::Float64,Y::Float64, R::Float64 )

  model = Model(
        optimizer_with_attributes(
            #Juniper.Optimizer,
            #"nl_solver" => optimizer_with_attributes(
                Ipopt.Optimizer, 
            #    MOI.Silent() => true,
            #),
        ),
    )
    set_optimizer_attribute(model, "print_level", 0)  ### Suppress Ipopt output
    set_optimizer_attribute(model, "tol", 1e-14)  ### Not required in general

    @variable(model, x )
    @variable(model, y )
    @variable(model, z >= 0)
  
    ### primal objective + Lagrange terms + augmented (squared penalty) terms
    @objective(model, Min, z+r[i] 
	+ vdual["x"]*(x + X - a[i]) + vdual["y"]*(y + Y - b[i]) + vdual["z"]*(z + r[i] - R)
	+ 0.5*rho*( (x+X-a[i])^2 + (y+Y-b[i])^2 + (z+r[i]-R)^2 ) )

    ### All coupling constraints are relaxed.
    @constraint( model, xyz, x^2 + y^2 <= z^2)
  
    #@show model
    
    #print(model)
    optimize!(model)

    return value(x), value(y), value(z)
end

function main()
  ## Initialization
    x = Dict{Int64,Float64}()
    y = Dict{Int64,Float64}()
    z = Dict{Int64,Float64}()

    ### Initialize Lagrangian multipliers to zero value
    vdual = Dict{Int64, Dict{String,Float64}}()
    for i in 1:NumVar
	vdual[i] = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    end

    X,Y,R=0.0,0.0,0.0
    oldX,oldY,oldR=0.0,0.0,0.0
    ### Initialize R to average of r[i], i=1,...,n
    for i in 1:length(a)
	R += (1.0/NumVar)*(r[i])
    end
  ## End of initialization

for n in 1:1000
    ### Solve AL subproblem for each subproblem i with X,Y,R fixed
    for i in 1:NumVar
      x[i],y[i],z[i] = solveSP(i,vdual[i],X,Y,R)
    end

    ### Subproblems in X,Y,R (with x,y,z fixed) can be solved in closed-form
    oldX,oldY,oldR=X,Y,R
    X,Y,R=0.0,0.0,0.0
    for i in 1:NumVar
	X += (1.0/NumVar)*(a[i]-x[i])
	Y += (1.0/NumVar)*(b[i]-y[i])
	R += (1.0/NumVar)*(z[i]+r[i])
    end

    ### Update Lagrange multipliers, compute a measure of primal infeasibility.
    discr = Dict("x"=>0.0,"y"=>0.0,"z"=>0.0)
    for i in 1:NumVar
	vdual[i]["x"] += rho*(x[i] + X - a[i]) 
	vdual[i]["y"] += rho*(y[i] + Y - b[i]) 
	vdual[i]["z"] += rho*(z[i] + r[i] - R)

	discr["x"] += abs(x[i] + X - a[i])
	discr["y"] += abs(y[i] + Y - b[i])
	discr["z"] += abs(z[i] + r[i] - R)
    end
    @printf("Iter %d Primal Discrepancies %8.8f R value %8.8f\n", 
	n, discr["x"] + discr["y"] + discr["z"], R)
    d_discr = abs(X-oldX)+abs(Y-oldY)+abs(R-oldR) ### Dual residues
    if discr["x"] + discr["y"] + discr["z"] + d_discr < 1e-7
	println("Stopping at iteration ",n)  ### Primal feasible within tolerance
	break
    end
end
    @show(X)
    @show(Y)
    @show(R)

println("Correct values are: ")

println("X = ",18.327253047203637)
println("Y = ",21.027195084962255)
println("R = ",44.62510960814807)
end

main()

Iter 1 Primal Discrepancies 84.21212302 R value 35.29791321
Iter 2 Primal Discrepancies 36.65309228 R value 43.88488364
Iter 3 Primal Discrepancies 36.43178281 R value 47.02848524
Iter 4 Primal Discrepancies 17.02880228 R value 48.99661811
Iter 5 Primal Discrepancies 21.70395209 R value 49.46616211
Iter 6 Primal Discrepancies 19.75454231 R value 48.60838825
Iter 7 Primal Discrepancies 2.47388693 R value 47.79193763
Iter 8 Primal Discrepancies 2.71413473 R value 47.16301155
Iter 9 Primal Discrepancies 2.50210965 R value 46.70696038
Iter 10 Primal Discrepancies 1.71755874 R value 46.36957820
Iter 11 Primal Discrepancies 0.69975570 R value 46.08054330
Iter 12 Primal Discrepancies 0.23853551 R value 45.77502759
Iter 13 Primal Discrepancies 0.88262004 R value 45.40853020
Iter 14 Primal Discrepancies 1.14502843 R value 44.96292089
Iter 15 Primal Discrepancies 1.72475055 R value 44.54451277
Iter 16 Primal Discrepancies 2.49358524 R value 44.25109528
Iter 17 Primal Discrepancies 2.69801978 R v

Iter 171 Primal Discrepancies 0.00000412 R value 44.62510961
Iter 172 Primal Discrepancies 0.00000396 R value 44.62510961
Iter 173 Primal Discrepancies 0.00000381 R value 44.62510961
Iter 174 Primal Discrepancies 0.00000366 R value 44.62510961
Iter 175 Primal Discrepancies 0.00000351 R value 44.62510961
Iter 176 Primal Discrepancies 0.00000338 R value 44.62510961
Iter 177 Primal Discrepancies 0.00000325 R value 44.62510961
Iter 178 Primal Discrepancies 0.00000312 R value 44.62510961
Iter 179 Primal Discrepancies 0.00000300 R value 44.62510961
Iter 180 Primal Discrepancies 0.00000288 R value 44.62510961
Iter 181 Primal Discrepancies 0.00000277 R value 44.62510960
Iter 182 Primal Discrepancies 0.00000266 R value 44.62510960
Iter 183 Primal Discrepancies 0.00000255 R value 44.62510960
Iter 184 Primal Discrepancies 0.00000245 R value 44.62510960
Iter 185 Primal Discrepancies 0.00000236 R value 44.62510960
Iter 186 Primal Discrepancies 0.00000227 R value 44.62510960
Iter 187 Primal Discrepa